In [1]:
import os

import pandas as pd 
import numpy as np

import math

#The data comes from http://www.tennis-data.co.uk/alldata.php
#I'm using the data from this year and the previous one (2020 and 2021)

In [2]:
#I read the xlsx files and merge them into a pandas dataframe
rawData = pd.concat([pd.read_excel("http://www.tennis-data.co.uk/2020/2020.xlsx"),pd.read_excel("http://www.tennis-data.co.uk/2021/2021.xlsx")])

In [3]:
#I create a new dataframe having only the original fields that matters and then I add new fields for interesting data
data = rawData[['Winner','Loser',"WRank","LRank"]]
data['Winrate winner'] = np.zeros(len(data))
data['Winrate loser'] = np.zeros(len(data))
data['Nb Game winner'] = np.zeros(len(data), dtype=int)
data['Nb Game loser'] = np.zeros(len(data), dtype=int)
data['Result win'] = np.zeros(len(data), dtype=int)
data['Result loose'] = np.zeros(len(data), dtype=int)

data

C:\Users\Megaport\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Megaport\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\Megaport\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.

,Winner,Loser,WRank,LRank,Winrate winner,Winrate loser,Nb Game winner,Nb Game loser,Result win,Result loose
0,Bublik A.,Mannarino A.,55.0,43.0,0.0,0.0,0,0,0,0
1,Moutet C.,Sandgren T.,81.0,68.0,0.0,0.0,0,0,0,0
2,Verdasco F.,Andujar P.,49.0,64.0,0.0,0.0,0,0,0,0
3,Bedene A.,Ymer M.,58.0,76.0,0.0,0.0,0,0,0,0
4,Chardy J.,Barrere G.,54.0,83.0,0.0,0.0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
735,Carreno Busta P.,Kwon S.W.,15.0,81.0,0.0,0.0,0,0,0,0
736,Alcaraz C.,Ruud C.,133.0,26.0,0.0,0.0,0,0,0,0
737,Carreno Busta P.,Ramos-Vinolas A.,15.0,47.0,0.0,0.0,0,0,0,0
738,Munar J.,Alcaraz C.,95.0,133.0,0.0,0.0,0,0,0,0


In [4]:
#I get the names of players winning and losing each game
winners = np.sort(pd.unique(data['Winner']))
losers = np.sort(pd.unique(data['Loser']))

#I concatenate the winners and loosers to get a single list
#I then erase the duplicated names and sort by alphabetic order the result
playersList = np.sort(pd.unique(np.concatenate((winners,losers), axis=None)))

#I create a new dataframe for the players
playersDf = pd.DataFrame({'Name': [], 'NbWins': [],'NbLooses': [],'NbGames': [],'Winrate': [],'Rank':[]})

for i in playersList:
    newPlayer = {'Name':i, 'NbWins':0, 'NbLooses':0, 'NbGames':0, 'Winrate':0, 'Rank':0}
    playersDf = playersDf.append(newPlayer, ignore_index=True)

playersDf

,Name,NbWins,NbLooses,NbGames,Winrate,Rank
0,Albot R.,0.0,0.0,0.0,0.0,0.0
1,Alcaraz C.,0.0,0.0,0.0,0.0,0.0
2,Altmaier D.,0.0,0.0,0.0,0.0,0.0
3,Anderson K.,0.0,0.0,0.0,0.0,0.0
4,Andreev A.,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
289,Zayid M.,0.0,0.0,0.0,0.0,0.0
290,Zeppieri G.,0.0,0.0,0.0,0.0,0.0
291,Zuk K.,0.0,0.0,0.0,0.0,0.0
292,Zverev A.,0.0,0.0,0.0,0.0,0.0


In [5]:
#I travel through all the games to get for each player of each matches
#- Did he win or loose ?
#- What was his rank at the time ? (the games being in a chronological order, the last rank is his current rank)

for i in range(len(data["Winner"])):
    winner = data.iloc[i]["Winner"]
    loser = data.iloc[i]["Loser"]
    
    playersDf.loc[playersDf['Name'] == winner, "NbGames"] += 1
    playersDf.loc[playersDf['Name'] == winner, "NbWins"] += 1
    
    
    playersDf.loc[playersDf['Name'] == loser, "NbGames"] += 1
    playersDf.loc[playersDf['Name'] == loser, "NbLooses"] += 1

    playersDf.loc[playersDf['Name'] == winner, "Rank"] = float(data.iloc[i]["WRank"])
    playersDf.loc[playersDf['Name'] == loser, "Rank"] = float(data.iloc[i]["LRank"])

for i in playersDf['Name']:
    playersDf.loc[playersDf['Name'] == i, "Winrate"] = playersDf.loc[playersDf['Name'] == i, "NbWins"]/playersDf.loc[playersDf['Name'] == i, "NbGames"]
    if math.isnan(float(playersDf.loc[playersDf['Name'] == i, "Rank"])):
        #If a player doesn't have a rank, I put him at the 1000 rank (I doesn't have many players without rank but one is enough to break the Neural Network)
        playersDf.loc[playersDf['Name'] == i, "Rank"] = float(1000)


for i in playersDf['Name']:
    winrate = float(playersDf.loc[playersDf['Name'] == i]["Winrate"])

#I then create the players file next to the clearData.ipynb file
playersDf.to_csv(r'players.csv', index = False)

playersDf

,Name,NbWins,NbLooses,NbGames,Winrate,Rank
0,Albot R.,12.0,16.0,28.0,0.428571,77.0
1,Alcaraz C.,7.0,6.0,13.0,0.538462,133.0
2,Altmaier D.,3.0,4.0,7.0,0.428571,143.0
3,Anderson K.,10.0,12.0,22.0,0.454545,91.0
4,Andreev A.,2.0,3.0,5.0,0.400000,423.0
...,...,...,...,...,...,...
289,Zayid M.,0.0,1.0,1.0,0.000000,1000.0
290,Zeppieri G.,0.0,2.0,2.0,0.000000,313.0
291,Zuk K.,0.0,1.0,1.0,0.000000,262.0
292,Zverev A.,37.0,11.0,48.0,0.770833,7.0


In [6]:
#After having collected the players' data, it's time to add those informations in each game (the winrate and the number of game of each player)
dataM = data.values
for i in dataM:
    winner = i[0]
    loser = i[1]
    
    if math.isnan(i[3]):
        i[3] = float(1000)
    if math.isnan(i[2]):
        i[2] = float(1000)

    #The rank value is 1/rank otherwise it deterriorates the learning process
    i[3] = 1/i[3]
    i[2] = 1/i[2]
    
    #Winrate winner
    i[4] = float(playersDf.loc[playersDf['Name'] == winner]["Winrate"])
    
    #Winrate looser
    i[5] = float(playersDf.loc[playersDf['Name'] == loser]["Winrate"])
    
    #Nb Game winner
    i[6] = float(playersDf.loc[playersDf['Name'] == winner]["NbGames"])
    
    #Nb Game looser
    i[7] = float(playersDf.loc[playersDf['Name'] == loser]["NbGames"])
    
    #Result win
    i[8] = 1
    
    
dataDf = pd.DataFrame(dataM,columns=['Winner','Loser','WRank','LRank','Winrate winner','Winrate loser','Nb Game winner','Nb Game loser','Result win','Result loose'])
#Note : In the raw data, the first player being always the winner, the last 2 colums are the same

#I create a file with the imteresting data of each game
dataDf.to_csv(r'dataCleared.csv', index = False)
dataDf

,Winner,Loser,WRank,LRank,Winrate winner,Winrate loser,Nb Game winner,Nb Game loser,Result win,Result loose
0,Bublik A.,Mannarino A.,0.0181818,0.0232558,0.529412,0.44186,51,43,1,0
1,Moutet C.,Sandgren T.,0.0123457,0.0147059,0.535714,0.37037,28,27,1,0
2,Verdasco F.,Andujar P.,0.0204082,0.015625,0.428571,0.291667,14,24,1,0
3,Bedene A.,Ymer M.,0.0172414,0.0131579,0.5,0.421053,36,19,1,0
4,Chardy J.,Barrere G.,0.0185185,0.0120482,0.538462,0.333333,26,15,1,0
...,...,...,...,...,...,...,...,...,...,...
2002,Carreno Busta P.,Kwon S.W.,0.0666667,0.0123457,0.65,0.5,40,26,1,0
2003,Alcaraz C.,Ruud C.,0.0075188,0.0384615,0.538462,0.609756,13,41,1,0
2004,Carreno Busta P.,Ramos-Vinolas A.,0.0666667,0.0212766,0.65,0.487179,40,39,1,0
2005,Munar J.,Alcaraz C.,0.0105263,0.0075188,0.428571,0.538462,21,13,1,0
